# AI Faireness 360 Python Demo

これは　
https://github.com/IBM/AIF360/blob/master/examples/tutorial_credit_scoring.ipynb
よりコピーし、日本語訳とaif360のpip installを加えたものです。

**AI Fairness 360 Open Source Toolkit**: https://aif360.mybluemix.net/

- API Doc: https://aif360.readthedocs.io/en/latest/index.html

# 与信決定に関する年齢バイアスの検出とマイグレーション
このチュートリアルの目的は、バイアスの検出とマイグレーションについて初心者の開発者にAI Fairness 360の基本機能を紹介することです。

## バイアスと機械学習
機械学習モデルは、特定のインスタンスの結果を予測します。 （ローン申請のインスタンスが与えられると、申請者がローンを返済するかどうかを予測します。）モデルは、他の多くのインスタンス（他のローン申請）および実際の結果（返済されるかどうか）が提供されるトレーニングデータセットに基づいてこれらの予測を行います。したがって、機械学習アルゴリズムは、新しいインスタンスの予測が必要なときに使用するトレーニングデータセット内のパターンまたは一般化を見つけようとします。 （たとえば、「給与が4万米ドルを超え、未払いの負債が5米ドル未満の場合、ローンを返済します」というパターンがあります。）多くのドメインで、この手法は教師付き機械学習と呼ばれ、非常にうまく機能しています。 。

ただし、検出されたパターンが望ましくない場合や、違法な場合もあります。たとえば、ローンの返済モデルでは、トレーニングデータセットが別の年齢グループよりも良い返済をしたため、年齢が返済の予測に重要な役割を果たすと判断する場合があります。これにより、2つの問題が発生します。1）トレーニングデータセットがすべての年齢層の真の人口を代表していない可能性があります。2）たとえ代表者であっても、これが履歴データに基づく適切な予測です。

AI Fairness 360は、公平性指標とバイアス緩和機能を使用して、この問題に対処するように設計されています。公平性メトリックを使用して、機械学習ワークフローのバイアスを確認できます。バイアス軽減機能を使用すると、ワークフローのバイアスを克服して、より公平な結果を得ることができます。

ローンのシナリオでは、違法なバイアスの直感的な例を説明しています。しかし、機械学習における望ましくない偏りがすべて違法であるわけではなく、より微妙な方法で存在する可能性もあります。たとえば、ローン会社は、すべての収入レベルにわたって顧客の多様なポートフォリオを必要とする可能性があり、したがって、低所得レベルよりも高所得レベルにより多くのローンを行っている場合は望ましくないとみなします。これは違法でも非倫理的でもありませんが、会社の戦略にとっては望ましくありません。

これらの2つの例が示すように、バイアス検出および/または緩和ツールキットは、関心のある特定のバイアスに合わせて調整する必要があります。より具体的には、保護属性と呼ばれる重要な属性を知る必要があります。Race(人種)は保護属性の一例であり、年齢は2番目です。

## 機械学習のワークフロー
バイアスが機械学習モデルに入る方法を理解するために、まず、教師あり機械学習プロセスでモデルを作成する方法の基本を確認します。

![image](https://github.com/kyokonishito/AIF360_jp/blob/master/examples/images/Complex_NoProc_V3.jpg?raw=true)

まず、プロセスは_training dataset_から始まります。これには、一連のインスタンスが含まれます。各インスタンスには、特徴とそれらの特徴の正しい予測という2つのコンポーネントがあります。次に、この学習データセットで機械学習アルゴリズムを学習して、機械学習モデルを作成します。この生成されたモデルは、新しいインスタンスが与えられたときに予測を行うために使用できます。 _テストデータセット_と呼ばれる、機能と正しい予測を含む2番目のデータセットを使用して、モデルの精度を評価します。このテストデータセットは、トレーニングデータセット、フィーチャと予測ペアのインスタンスのセットと同じ形式であるため、多くの場合、これら2つのデータセットは同じ初期データセットから派生します。ランダムパーティションアルゴリズムを使用して、初期データセットをトレーニングデータセットとテストデータセットに分割します。

バイアスは、上記の3つのステップのいずれかでシステムに入ることができます。トレーニングデータセットは、その結果が特定の種類のインスタンスに偏っている可能性があるという点で偏っている場合があります。モデルを作成するアルゴリズムは、入力内の特定の特徴に重み付けされたモデルを生成する可能性があるという点で偏っている場合があります。テストデータセットは、バイアスがかけられる可能性がある正解に対する期待があるという点で、バイアスがかけられる場合があります。機械学習プロセスのこれら3つのポイントは、バイアスをテストおよび緩和するためのポイントを表しています。 AI Fairness 360コードベースでは、これらのポイントをpre-processing _、_ in-processing_、および_post-processingと呼びます。

## AI Fairness 360
これで、AI Fairness 360（aif360）を活用してバイアスを検出および軽減する準備が整いました。ドイツのクレジットデータセットを使用して、トレーニングデータセットとテストデータセットに分割します。典型的なクレジットアプリケーションのさまざまな機能に基づいて、申請者にクレジットを付与する必要があるかどうかを予測するために、機械学習モデルの作成におけるバイアスを探します。 protected属性は「Age」で、「1」（25以上）と「0」（25歳未満）がそれぞれ特権グループと非特権グループの値です。
この最初のチュートリアルでは、初期トレーニングデータのバイアスをチェックし、バイアスを緩和してから再チェックします。より高度な機械学習ワークフローは、著者のチュートリアルとデモで提供されています

以下が手順になります:
#### Step 1:  ライプラリの導入とインポート文の記述　
#### Step 2: 保護された属性を指定してデータセットを読み込み、データセットをtrainとtestに分割します
#### Step 3: 元のトレーニングデータセットの公平性メトリックを計算する
#### Step 4: 元のデータセットを変換してバイアスを軽減する
#### Step 5: 変換されたデータセットの公平性メトリックを計算する

### Step 1 ライプラリの導入とインポート文の記述　
(githubオリジナル版にはないですが、ライブラリの導入を最初にpipコマンドで行なっています) 

他の python プログラムと同様に、最初のステップは必要なパッケージをインポートすることです。 以下では、`aif360`パッケージからいくつかのコンポーネントをインポートします。 GermanDataset、バイアスをチェックするためのメトリクス、バイアスを緩和するために使用するアルゴリズムに関連するクラスをインポートします。

In [ ]:
#githubオリジナル版にはないですが、ライブラリの導入が事前に必要です。　
# tensorflow と1autoai-libsのERRORは今回使用しないので無視してください。
!pip install aif360

In [ ]:
# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

### Step2 保護された属性を指定してデータセットを読み込み、データセットをtrainとtestに分割します

ステップ2では、保護属性をageに設定して、初期データセットをロードします。 次に、元のデータセットをトレーニングデータセットとテストデータセットに分割します。 このチュートリアルではトレーニングデータセットのみを使用しますが、通常のワークフローではテストデータセットを使用して、機械学習モデルの開発中に有効性（精度、公平性など）を評価します。 最後に、年齢属性の特権値（1）と非特権値（0）に2つの変数（ステップ3で使用）を設定します。 これらは、バイアスを検出および軽減するための重要な入力であり、ステップ3およびステップ4になります。

In [ ]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

### Step 3 元のトレーニングデータセットの公平性メトリックを計算する
保護属性「年齢」を特定し、特権値と非特権値を定義したので、aif360を使用してデータセットのバイアスを検出できます。 1つの簡単なテストは、特権グループと非特権グループの好ましい結果の割合を比較し、前者の割合を後者から差し引くことです。 負の値は、特権のないグループにとって好ましくない結果を示します。 これは、BinaryLabelDatasetMetricクラスのmean_differenceというメソッドで実装されます。 以下のコードはこのチェックを実行し、出力を表示して、差が-0.169905であることを示しています。

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

### Step4 元のデータセットを変換してバイアスを軽減する
前のステップでは、特権グループがトレーニングデータセットで17％多くの肯定的な結果を得ていることが示されました。これは望ましくないため、トレーニングデータセットのこのバイアスを軽減しようとします。前述のように、これはモデルの作成前に発生するため、_pre-processing_ mitigationと呼ばれます。

AI Fairness 360は、いくつかの前処理緩和アルゴリズムを実装しています。 `aif360.algorithms.preprocessing`パッケージの` Reweighing`クラスに実装されているReweighingアルゴリズム[1]を選択します。このアルゴリズムは、データセットを変換して、特権グループと非特権グループの保護された属性の肯定的な結果がより公平になるようにします。

次に、fitメソッドとtransformメソッドを呼び出して変換を実行し、新しく変換されたトレーニングデータセット（dataset_transf_train）を生成します。

`[1] F. Kamiran and T. Calders,  "Data Preprocessing Techniques for Classification without Discrimination," Knowledge and Information Systems, 2012.`

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

### Step 5 変換されたデータセットの公平性メトリックを計算する
変換されたデータセットができたので、ステップ3で元のトレーニングデータセットに使用したものと同じメトリックを使用して、バイアスを除去することの効果を確認できます。もう一度、BinaryLabelDatasetMetricクラスで関数mean_differenceを使用します。 緩和ステップは非常に効果的であり、平均結果の差は0.0になりました。 そのため、特権グループの17％の優位性から、平均結果の点で平等になりました。

In [ ]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

### まとめ
このチュートリアルの目的は、新しいユーザーにバイアスの検出と緩和を提供し、AI Fairness 360の機能の一部を穏やかに紹介することです。より完全なユースケースでは、次のステップに進み、変換されたデータセットが精度と公平性に与える影響を確認します訓練されたモデルの。これは、demo_reweighing_preproc.ipynbと呼ばれるツールキットのexamplesディレクトリのデモノートブックに実装されています。このノートブックは、この簡単なチュートリアルの一般化と応用であるため、読者の皆様にはぜひご覧ください。

バイアスの存在を検出するために使用できる多くのメトリックがあります。 AI Fairness 360は、それらの多くを使用するために提供します。これらのメトリックのどれを使用するかが明確ではないため、いくつかのガイダンスも提供します。同様に、使用できるさまざまなバイアス緩和アルゴリズムがあり、その多くはAI Fairness 360にあります。他のチュートリアルでは、これらのメトリックと緩和アルゴリズムのいくつかの使用方法を示します。

前述のように、機械学習パイプラインのさまざまな段階で、公平性メトリックと軽減アルゴリズムの両方を実行できます。アプリケーションドメインに関連するメトリックをできるだけ多く使用して、できるだけ頻繁にバイアスをチェックすることをお勧めします。また、ソフトウェアプロジェクトの進化に伴うバイアス認識を確保するために、自動化された継続的統合パイプラインにバイアス検出を組み込むことをお勧めします。